# Step 1: Package Installation
First we import the necessary packages

In [1]:
import or_suite
import numpy as np

import copy

import os
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd


import gym

# Step 2: Pick problem parameters for the environment

Here we use the ambulance metric environment as outlined in `or_suite/envs/ambulance/ambulance_metric.py`.  The package has default specifications for all of the environments in the file `or_suite/envs/env_configs.py`, and so we use one the default for the ambulance problem in a metric space.

In addition, we need to specify the number of episodes for learning, and the number of iterations (in order to plot average results with confidence intervals).

In [2]:
CONFIG =  or_suite.envs.env_configs.airline_default_config

epLen = CONFIG['epLen']
nEps = 1
numIters = 100

# Step 3: Pick simulation parameters

Next we need to specify parameters for the simulation. This includes setting a seed, the frequency to record the metrics, directory path for saving the data files, a deBug mode which prints the trajectory, etc.

In [3]:
DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, 
                    'epLen' : 5,
                    'render': False,
                    'pickle': False
                    }


revenue_env = gym.make('Airline-v0', config=CONFIG)
mon_env = Monitor(revenue_env)

# Step 4: Pick list of algorithms

We have several heuristics implemented for each of the environments defined, in addition to a `Random` policy, and some `RL discretization based` algorithms. 

The `Stable` agent only moves ambulances when responding to an incoming call and not in between calls. This means the policy $\pi$ chosen by the agent for any given state $X$ will be $\pi_h(X) = X$

The `Median` agent takes a list of all past call arrivals sorted by arrival location, and partitions it into $k$ quantiles where $k$ is the number of ambulances. The algorithm then selects the middle data point in each quantile as the locations to station the ambulances.

In [4]:
agents = { # 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
'Random': or_suite.agents.rl.random.randomAgent(),
'BayesSelector': or_suite.agents.airline_revenue_management.bayes_selector.bayes_selectorAgent(epLen),
'BayesSelectorBadRounding': or_suite.agents.airline_revenue_management.bayes_selector.bayes_selectorAgent(epLen, round_flag=False),
}

# Step 5: Run Simulations

Run the different heuristics in the environment

In [5]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/airline_'+str(agent)
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(revenue_env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/airline_'+str(agent))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/airline_'+str(agent))
        algo_list_radar.append(str(agent))

Random
**************************************************
Running experiment
**************************************************
**************************************************
Experiment complete
**************************************************
**************************************************
Saving data
**************************************************
Writing to file data.csv
**************************************************
Data save complete
**************************************************
BayesSelector
**************************************************
Running experiment
**************************************************
**************************************************
Experiment complete
**************************************************
**************************************************
Saving data
**************************************************
Writing to file data.csv
**************************************************
Data save complete
************************

In [6]:
fig_path = '../figures/'
fig_name = 'revenue'+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)

# 
additional_metric = {}
fig_name = 'revenue'+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)



                  Algorithm  Reward      Time     Space
0                    Random    1.55  7.531308  -4025.27
1             BayesSelector    2.79  3.622935 -32864.49
2  BayesSelectorBadRounding    2.69  3.645927 -29529.85


### Dual Degeneracy Examples

In [7]:
p = .45 # either do .44 or .45
CONFIG['P'] = np.asarray([[1-p, p],[1-p,p],[1-p,p],[1-p,p],[1-p,p]])

DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, 
                    'epLen' : 5,
                    'render': False,
                    'pickle': False
                    }


revenue_env = gym.make('Airline-v0', config=CONFIG)
mon_env = Monitor(revenue_env)

agents = { # 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
'Random': or_suite.agents.rl.random.randomAgent(),
'BayesSelector': or_suite.agents.airline_revenue_management.bayes_selector.bayes_selectorAgent(epLen),
'BayesSelectorBadRounding': or_suite.agents.airline_revenue_management.bayes_selector.bayes_selectorAgent(epLen, round_flag=False),
}

path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/airline_'+str(agent)
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(revenue_env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/airline_'+str(agent))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/airline_'+str(agent))
        algo_list_radar.append(str(agent))
        
        
fig_path = '../figures/'
fig_name = 'revenue'+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)

# 
additional_metric = {}
fig_name = 'revenue'+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)






Random
**************************************************
Running experiment
**************************************************
**************************************************
Experiment complete
**************************************************
**************************************************
Saving data
**************************************************
Writing to file data.csv
**************************************************
Data save complete
**************************************************
BayesSelector
**************************************************
Running experiment
**************************************************
**************************************************
Experiment complete
**************************************************
**************************************************
Saving data
**************************************************
Writing to file data.csv
**************************************************
Data save complete
************************

# Run with different parameters

In [8]:
epLen = 4
A = np.asarray([[1, 1, 0,0,0,0], [ 0,0, 1, 1, 1, 1], [ 0,0, 0,0, 1, 1] ])
tau = 23
P = np.ones((tau, A.shape[1]))/3
c = [5, 5, 5]
f = range(10, 16)
CONFIG = {'A': A, 'f': f, 'P': P, 'starting_state': c , 'tau': tau}
nEps = 134
numIters = 100

In [9]:
m = 6
l = 3
A = np.identity(m)
for i in range(l):
    for j in range(l):
        if i != j:
            demand_col = np.zeros((m, 1))
            demand_col[2 * i + 1] = 1.0
            demand_col[2 * j] = 1.0
            A=  np.append(A, demand_col, axis = 1)
A = np.append(A, A, axis = 1)
tau = 20
P = np.array([0.01327884, 0.02244177, 0.07923761, 0.0297121,  0.02654582, 0.08408091, 0.09591975, 0.00671065, 0.08147508, 0.00977341, 0.02966204, 0.121162, 0.00442628, 0.00748059, 0.02641254, 0.00990403, 0.00884861, 0.02802697, 0.03197325, 0.00223688, 0.02715836, 0.0032578,  0.00988735, 0.04038733])
P = np.array([P]*tau)
c = [2]*6
f = np.array([33, 28, 36, 34, 17, 20, 39, 24, 31, 19, 30, 48, 165, 140, 180, 170, 85, 100,195, 120, 155, 95, 150, 240])
CONFIG = {'epLen':epLen, 'A': A, 'f': f, 'P': P, 'starting_state': c , 'tau': tau}
epLen = CONFIG['epLen']


In [10]:
DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, 
                    'epLen' : 5,
                    'render': False,
                    'pickle': False
                    }


revenue_env = gym.make('Airline-v0', config=CONFIG)
mon_env = Monitor(revenue_env)

In [11]:
agents = { # 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
'Random': or_suite.agents.rl.random.randomAgent(),
'BayesSelector': or_suite.agents.airline_revenue_management.bayes_selector.bayes_selectorAgent(epLen),
'BayesSelectorBadRounding': or_suite.agents.airline_revenue_management.bayes_selector.bayes_selectorAgent(epLen, round_flag=False),
}

In [ ]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/airline_'+str(agent)
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(revenue_env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/airline_'+str(agent))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/airline_'+str(agent))
        algo_list_radar.append(str(agent))

Random
**************************************************
Running experiment
**************************************************
**************************************************
Experiment complete
**************************************************
**************************************************
Saving data
**************************************************
Writing to file data.csv
**************************************************
Data save complete
**************************************************
BayesSelector
**************************************************
Running experiment
**************************************************


In [ ]:
fig_path = '../figures/'
fig_name = 'revenue'+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)

# 
additional_metric = {}
fig_name = 'revenue'+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

